In [ ]:
import os
import sys
sys.path.append("../")
import yaml
import json
import numpy as np
import torch
import math
import copy
import re
from transformers import AutoModel, AutoTokenizer
########Pretrain model#################
# ckpt_path = '/yourpath/songcomposer_pretrain'
########SFT model#################
# ckpt_path = '/yourpath/songcomposer_sft'
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
model = AutoModel.from_pretrained(ckpt_path, trust_remote_code=True).cuda().half()

**Format Notation**

\<bop\>Paired data\<eop\>. \<bop\> stands for the **b**eginning **o**f the **p**air. \<eop\> stands for the **e**nd **o**f the **p**air.

\<bom\>Pure melody\<eom\>. \<bom\> stands for the **b**eginning **o**f the **m**elody. \<eom\> stands for the **e**nd **o**f the **m**elody.

\<bol\>Pure lyric\<eol\>. \<bol\> stands for the **b**eginning **o**f the **l**yric. \<eol\> stands for the **e**nd **o**f the **l**yric.

The conversation format would be:

[UNUSED_TOKEN_146]user\n**Question**[UNUSED_TOKEN_145]\n

[UNUSED_TOKEN_146]assistant\n**Answer**[UNUSED_TOKEN_145]\n

In [ ]:
####### For pretrain model ########
####### Test continuation ability#######
####### pair data #######
prompt = '<bop> Total 7 lines. The first line:可,<D4>,<137>,<79>|惜,<D#4>,<137>,<79>|这,<F4>,<137>,<88>|是,<F4>,<121>,<79>|属,<F4>,<121>,<79>|于,<D#4>,<214>,<88>|你,<D#4>,<141>,<79>|的,<D4>,<130>,<79>|风,<C4>,<151>,<79>|景,<A#3> <F3>,<181><137>,<79>\n'
####### pure melody #######
prompt = '<bom> Total 3 lines. The first line:<F4>,<106>,<79>|<F4>,<106>,<79>|<F4>,<106>,<88>|<F4>,<116>,<88>|<F4>,<100>,<79>|<F4>,<100>,<88>|<F4>,<85>,<79>|<F4>,<85>,<79>|<F4>,<85>,<79>\n'
####### pure lyric #######
prompt = '<bol> Total 10 lines. The first line:阳|光|打|破|了|天|空|的|沉|寂\n'
###### Inference function would generation a three-shot answer. Find the best fit one.##########
model.inference_pretrain(prompt, tokenizer)

In [ ]:
####### For sft model ########
####### l2m #######
prompt = 'Compose a tune in harmony with the accompanying lyrics. <bol> Total 6 lines.\
The first line:轻|轻|笑|声|在|为|我|送|温|暖\n\
The second line:你|为|我|注|入|快|乐|强|电\n\
The third line:轻|轻|说|声|漫|长|路|快|要|走|过\n\
The fourth line:终|于|走|到|明|媚|晴|天\n\
The fifth line:声|声|欢|呼|跃|起|像|红|日|发|放|金|箭\n\
The sixth line:我|伴|你|往|日|笑|面|重|现\n<eol>'
####### m2l #######
prompt = 'Create lyrics to accompany the given melody. <bom> Total 7 lines.\
The first line:<D4>,<141>,<79>|<F4>,<151>,<79>|<G4>,<172>,<79>|<A4>,<147>,<112>|<D4>,<147>,<79>|<G4>,<166>,<79>|<A4>,<172>,<79>|<C5>,<144>,<79>|<A4>,<268>,<212>\n\
The second line:<D4>,<141>,<79>|<F4>,<154>,<79>|<G4>,<169>,<79>|<A4>,<144>,<108>|<D4>,<151>,<79>|<G4>,<151>,<137>\n\
The third line:<E4>,<151>,<88>|<E4>,<141>,<79>|<F4>,<144>,<79>|<E4>,<137>,<79>|<F4>,<141>,<79>|<G4>,<130>,<79>|<A4>,<157>,<79>\n\
The fourth line:<G4>,<144>,<79>|<A4>,<137>,<79>|<D4>,<160>,<141>|<G4>,<144>,<79>|<A4>,<134>,<79>|<D4>,<200>,<79>|<C4>,<194>,<104>|<D4>,<264>,<154>\n\
The fifth line:<A4>,<141>,<79>|<C5>,<147>,<79>|<A4>,<157>,<79>|<G4>,<224>,<92>|<A4>,<151>,<79>|<G4> <F4>,<189><134>,<79>|<D4>,<197>,<79>|<A4>,<233>,<79>\n\
The sixth line:<A4>,<144>,<79>|<B4>,<141>,<79>|<A4>,<154>,<79>|<G4>,<237>,<79>|<A4>,<137>,<79>|<G4>,<120>,<79>|<G4>,<120>,<144>|<D5>,<233>,<96>|<A4>,<226>,<154>\n\
The seventh line:<G4> <A4>,<154><104>,<79>|<C5>,<175>,<79>|<D5>,<137>,<79>|<C5>,<151>,<79>|<A4>,<151>,<79>|<C5>,<151>,<79>|<G4>,<137>,<154>\n<eom>'
####### song cont #######
prompt = 'Continue the existing song script by adding 2 additional lines. <bop> Total 2 lines.\
The first line:当,<B3>,<147>,<79>|春,<D#4>,<144>,<79>|天,<E4>,<157>,<144>|为,<E4> <F#4>,<134><147>,<88>|我,<F#4>,<137>,<88>|冒,<F#4>,<147>,<79>|着,<B4>,<197>,<79>|雨,<D#4>,<144>,<79>|到,<F#4>,<157>,<79>|达,<E4>,<160>,<79>\n\
The second line:只,<E4>,<116>,<116>|为,<E4> <E4>,<147><144>,<79>|了,<G#4> <A4>,<134><154>,<79>|带,<G#4> <F#4>,<88><108>,<79>|我,<G#4> <F#4>,<127><134>,<79>|去,<E4>,<151>,<79>|往,<C#4>,<202>,<235>\n<eop>'
####### text-to-song #######
prompt = 'Create a song on brave and sacrificing with a rapid pace.'
###### Inference function would generation a three-shot answer. Find the best fit one.##########
model.inference(prompt, tokenizer)

In [ ]:
######We provide the code for parsing the string-like song into the MIDI file and lyric txt.
line = 'The first line:勇, <E4> , <154> , <88> |敢, <E4> , <134> , <88> |地, <E4> , <137> , <79> |去, <F#4> , <151> , <79> |相, <E4> , <154> , <79> |信, <D#4> , <154> , <79> |自, <C#4> , <157> , <79> |己, <B3> , <172> , <127> The second line:坚, <E4> , <151> , <88> |定, <E4> , <137> , <88> |地, <E4> , <137> , <79> |向, <F#4> , <151> , <79> |前, <E4> , <151> , <79> |奔, <D#4> , <160> , <79> |跑, <C#4> , <157> , <79> The third line:哪, <B3> , <151> , <79> |里, <G#3> , <137> , <79> |会, <B3> , <151> , <79> |有, <G#3> , <189> , <79> |风, <F#3> , <157> , <79> |雨, <G#3> , <137> , <79> The fourth line:不, <G#3> , <147> , <79> |回, <F#3> , <144> , <79> |头, <E3> , <151> , <79> |的, <F#3> , <141> , <79> |笑, <G#3> , <166> , <79> |着, <B3> , <219> , <160> The fifth line:每, <E4> , <154> , <88> |一, <E4> , <130> , <88> |个, <E4> , <144> , <79> |起, <F#4> , <147> , <79> |点, <E4> , <157> , <79> |都, <D#4> , <154> , <79> |是, <C#4> , <151> , <79> |我, <B3> , <118> , <79> |们, <B3> , <118> , <79> |成, <G#3> , <207> , <79> |功, <B3> , <205> , <79> |的, <G#3> , <205> , <79>'
from finetune.utils import gen_midi
gen_midi(line, 'text')